In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd
from utils import specificity_score, negative_prediction_value_score, gmean_score, informedness_score
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from base_pulsa_ai import BasePulsarAI
import pennylane as qml

In [39]:
base_pulsar_ai = BasePulsarAI()
x_train, x_test, y_train, y_test = base_pulsar_ai.get_torch_htru_2()

In [45]:
wires = range(8)

def conv(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    
def conv_layer(params, stride, n_qubits):
    for i in range(n_qubits):
        conv(params[2*i:2*i+2], [wires[i], wires[(i+stride)%n_qubits]])

n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    #conv_layer(weights, 5, 8)

    #qml.Barrier()
    #conv_layer(weights, 3, 4)

    #qml.Barrier()

    return qml.expval(qml.PauliZ(wires=n_qubits-1))


random_input = np.random.randn(8)
weights = np.random.randn(16)

#qml.draw_mpl(qnode)(random_input, weights)

In [46]:
class QuantumCNN(nn.Module):
    def __init__(self, num_features, output_dim):
        super(QuantumCNN, self).__init__()
        self.qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

    def forward(self, x):
        x = x.view(-1, 8, 1)
        
        x = self.qlayer(x)
        return x


input_dim  = 8 # number of features
output_dim = 2 # binary classification, to be change for sigmoid ?


model = QuantumCNN(input_dim, output_dim)
output_train = model(x_train)

print(output_train)


tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]])


In [47]:


ss = ShuffleSplit(n_splits=base_pulsar_ai.num_runs, train_size=60, test_size=120)

def train_network(model,optimizer,criterion,x_train,y_train,x_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(x_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train)
        
        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()
        
        output_test = model(x_test)
        loss_test = criterion(output_test,y_test)

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

        print(f"Epoch {epoch} train loss: {loss_train.item()} test loss: {loss_test.item()}")

for train_index, test_index in ss.split(x_train):
    num_epochs = base_pulsar_ai.max_num_epochs
    train_losses = np.zeros(num_epochs)
    test_losses  = np.zeros(num_epochs)
    model = QuantumCNN(input_dim, output_dim)

    learning_rate = 0.01
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

    train_network(model,optimizer,criterion, x_train,y_train, x_test,y_test,num_epochs,train_losses,test_losses)

    # Predict the test set
    output_test = model(x_test)
    _, predicted_test = torch.max(output_test, 1)

    # Calculate the scores
    base_pulsar_ai.append_score(y_test, predicted_test)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
# Print the scores
for metric, values in base_pulsar_ai.scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")